In [ ]:
import numpy as np
import math
import pandas as pd
import torch
import torch.nn as nn
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#loading pretrained model

In [ ]:
import gensim.downloader as api
w2v_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


#Defining text preprocessing class

In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.punctuations = set(string.punctuation)

    def process_text(self, text):
        text = text.lower()
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'<.*?>', '', text)
        text = ''.join(char for char in text if char not in self.punctuations)
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in self.stop_words]
        text = ' '.join(tokens)

        return text

    def process_column(self, df, column_name):
        df[column_name] = df[column_name].apply(self.process_text)

        return df

#Loading dataset

In [ ]:
data = pd.read_csv('restuarents.csv', encoding='ISO-8859-1')
print(data.head())

                                                text  label
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1


#Applying text preprocessor

In [ ]:
preprocessor = TextPreprocessor()
df = preprocessor.process_column(data, 'text')

print(df.head())

                                                text  label
0                                    wow loved place      1
1                                         crust good      0
2                                tasty texture nasty      0
3  stopped late may bank holiday rick steve recom...      1
4                        selection menu great prices      1


#Function for getting w2v representations

In [ ]:
def w2v(sentence):
    tokenized_data = sentence.split()
    n_tokens = len(tokenized_data)
    if n_tokens >= 10:
        tokenized_data = tokenized_data[:10]
    else:
        pad_length = 10 - n_tokens
        tokenized_data += ["<EOS>"] * pad_length

    vectors = []
    for token in tokenized_data:
        if token in w2v_model:
            vec = w2v_model[token]
        else:
            vec = np.zeros(w2v_model.vector_size)
        vectors.append(vec)

    tensor = torch.stack([torch.tensor(vec, dtype=torch.float32) for vec in vectors])
    return tensor

#Testing w2v function

In [ ]:
t = w2v("I am a good student")
print(t.shape)

g = w2v("Hey are you going to the zoo tomorrow morning with me and my friends")
print(g.shape)

y = w2v("you are a fool")

f =w2v("I quit")
print(y.shape)
print(f.shape)

torch.Size([10, 300])
torch.Size([10, 300])
torch.Size([10, 300])
torch.Size([10, 300])


#Defining Datasethelper child class

In [ ]:
class Datasethelper(Dataset):
  def __init__(self, df):
    super().__init__()
    self.data = df['text'].values
    self.labels = df['label'].values

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    text = self.data[index]
    label = self.labels[index]
    w2v_data = w2v(text)
    label = torch.tensor( label , dtype=torch.float32)
    #print(w2v_data.shape)
    #print(w2v_data.dtype)
    return w2v_data, label

#Splitting, preparing and loading dataset

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_helper = Datasethelper(train_df)
test_helper = Datasethelper(test_df)


train_dloader = DataLoader(train_helper, batch_size = 12 , shuffle = True)
test_dloader = DataLoader(test_helper, batch_size = 12 , shuffle = False)


In [ ]:
for x,y in train_dloader:
  break
print(x.shape)
print(y.shape)

torch.Size([12, 10, 300])
torch.Size([12])


#Defining NN model

In [ ]:
class NN4(nn.Module):
    def __init__(self, input_dim = 300, hidden_dim1 = 128, hidden_dim2 = 64):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = 1
        self.LL1 = nn.Linear(self.input_dim, self.hidden_dim1)
        self.LL2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.FL = nn.Linear(self.hidden_dim2, self.output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        X=torch.mean(X,1)
        X.requires_grad=True
        X = self.LL1(X)
        X = self.sigmoid(X)
        X = self.LL2(X)
        X = self.sigmoid(X)
        X = self.FL(X)
        X = self.sigmoid(X)

        return X

#Initializing model, optimizer and criterion

In [ ]:
NNmodel = NN4()
criterion=nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(NNmodel.parameters(), lr = 0.001)

#Training

In [ ]:
for epoch in range(10):
  overall_accuracy=0
  for x,y in train_dloader:
    #print("from tloader x :" + str(x.shape))
    #print("from tloader y :" + str(y.shape))
    predicted_y=NNmodel(x)
    #print("NNmodel out y_pred:" +str(predicted_y.shape))
    batch_size=x.shape[0]
    y=y.view(batch_size,1)
    #print("tloader y from view:"+ str(y.shape))
    loss=criterion(predicted_y,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_true=y.detach().numpy()
    #y_pred=predicted_y.detach().numpy() >0.5
    y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
    accuracy= accuracy_score(y_true,y_pred)
    overall_accuracy +=accuracy*batch_size


  print(f'Epoch: {epoch} --> Accuracy {overall_accuracy/len(train_helper)}')



Epoch: 0 --> Accuracy 0.8657534246575342
Epoch: 1 --> Accuracy 0.8561643835616438
Epoch: 2 --> Accuracy 0.8671232876712329
Epoch: 3 --> Accuracy 0.863013698630137
Epoch: 4 --> Accuracy 0.8643835616438356
Epoch: 5 --> Accuracy 0.863013698630137
Epoch: 6 --> Accuracy 0.8684931506849315
Epoch: 7 --> Accuracy 0.8753424657534247
Epoch: 8 --> Accuracy 0.8753424657534247
Epoch: 9 --> Accuracy 0.873972602739726


#Testing

In [ ]:
test_accuracy = 0
overall_accuracy = 0
for x, y in test_dloader:
    predicted_y = NNmodel(x)
    batch_size = x.shape[0]
    y = y.view(batch_size, 1)
    y_true = y.detach().numpy()
    y_pred = predicted_y.detach().numpy() > 0.5
    accuracy = accuracy_score(y_true, y_pred)
    overall_accuracy += accuracy * batch_size
test_accuracy = overall_accuracy / len(test_helper)
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.8469945355191257


#Testing per epoch

In [ ]:
test_accuracy = 0
for epoch in range(10):
    overall_accuracy = 0
    for x, y in test_dloader:
        predicted_y = NNmodel(x)
        batch_size = x.shape[0]
        y = y.view(batch_size, 1)
        y_true = y.detach().numpy()
        y_pred = predicted_y.detach().numpy() > 0.5
        accuracy = accuracy_score(y_true, y_pred)
        overall_accuracy += accuracy * batch_size
    test_accuracy = overall_accuracy / len(test_helper)
    print(f'Test Epoch: {epoch} --> Accuracy {test_accuracy}')

Test Epoch: 0 --> Accuracy 0.8360655737704918
Test Epoch: 1 --> Accuracy 0.8360655737704918
Test Epoch: 2 --> Accuracy 0.8360655737704918
Test Epoch: 3 --> Accuracy 0.8360655737704918
Test Epoch: 4 --> Accuracy 0.8360655737704918
Test Epoch: 5 --> Accuracy 0.8360655737704918
Test Epoch: 6 --> Accuracy 0.8360655737704918
Test Epoch: 7 --> Accuracy 0.8360655737704918
Test Epoch: 8 --> Accuracy 0.8360655737704918
Test Epoch: 9 --> Accuracy 0.8360655737704918


#RNN

Rnn1- problem : output dim changes in the end

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=64, num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, X):
        _, hn = self.rnn(X)
        return hn

Rnn2

In [ ]:
class RNN2(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=64, num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, X):
        batch_size = X.size(0)  # Get the batch size
        _, hn = self.rnn(X)
        hn = hn.permute(1, 0, 2).contiguous()  # Transpose and reshape to [batch_size, 1, hidden_dim]
        return hn

Passing hn to LL via NN5- problem y and pred_y Dim don't match

In [ ]:
class NN5(nn.Module):
    def __init__(self, input_dim = 64, hidden_dim1 = 128, hidden_dim2 = 64):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = 1
        self.LL1 = nn.Linear(self.input_dim, self.hidden_dim1)
        self.LL2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.FL = nn.Linear(self.hidden_dim2, self.output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        batch_size = X.shape[0]
        X=torch.mean(X,1)
        #X.requires_grad=True
        X = torch.tensor(X, requires_grad=True)
        X = self.LL1(X)
        X = self.sigmoid(X)
        X = self.LL2(X)
        X = self.sigmoid(X)
        X = self.FL(X)
        #X = self.sigmoid(X)

        return X

Passing hn to LL via NN6

In [ ]:
class NN6(nn.Module):
    def __init__(self, input_dim=64, hidden_dim1=128, hidden_dim2=64, output_dim=1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.LL1 = nn.Linear(self.input_dim, self.hidden_dim1)
        self.LL2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.FL = nn.Linear(self.hidden_dim2, self.output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        batch_size = X.shape[0]
        X = torch.mean(X, 1)
        X = self.LL1(X)
        X = self.sigmoid(X)
        X = self.LL2(X)
        X = self.sigmoid(X)
        X = self.FL(X)
        X = self.sigmoid(X)

        #return X
        return X.view(batch_size, self.output_dim)


#Initializing the models, setting optimer and criterion

In [ ]:
rnn_model = RNN2()
nn_model2 = NN6()

criterion2 = nn.BCELoss(reduction='mean')
optimizer2 = torch.optim.Adam(nn_model2.parameters(), lr=0.001)



Checking shapes of the outputs from the models

In [ ]:
for t,p in train_dloader:
  rnnout = rnn_model(t)
  print(rnnout.shape)
  g = nn_model2(rnnout)
  print(g.shape)


torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Size([12, 1])
torch.Size([12, 1, 64])
torch.Si

#Training

In [ ]:
for epoch in range(10):
    overall_accuracy = 0
    for x, y in train_dloader:
        #print(f'from tloader y: {y.shape}')
        rnn_output = rnn_model(x)
        #print(f'rnn out:  {rnn_output.shape}')
        predicted_y = nn_model2(rnn_output)
        #print(f'pred-y from NN6: {predicted_y.shape}')
        batch_size = x.shape[0]
        #predicted_y = predicted_y.squeeze(1).t()
        #print(f'pred-y from squeeze: {predicted_y.shape}')
        y = y.view(batch_size, 1)
        #print(f'final y from view: {y.shape}')
        loss = criterion2(predicted_y, y)
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()
        y_true = y.detach().numpy()
        y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
        accuracy = accuracy_score(y_true, y_pred)
        overall_accuracy += accuracy * batch_size

    print(f'Epoch: {epoch} --> Accuracy {overall_accuracy/len(train_helper)}')


Epoch: 0 --> Accuracy 0.5150684931506849
Epoch: 1 --> Accuracy 0.5315068493150685
Epoch: 2 --> Accuracy 0.5315068493150685
Epoch: 3 --> Accuracy 0.5191780821917809
Epoch: 4 --> Accuracy 0.5246575342465754
Epoch: 5 --> Accuracy 0.5342465753424658
Epoch: 6 --> Accuracy 0.5589041095890411
Epoch: 7 --> Accuracy 0.5260273972602739
Epoch: 8 --> Accuracy 0.5328767123287671
Epoch: 9 --> Accuracy 0.5383561643835616


#Testing

In [ ]:
test_accuracy = 0
overall_accuracy = 0
for x, y in test_dloader:
    rnn_output = rnn_model(x)
    predicted_y = nn_model2(rnn_output)
    batch_size = x.shape[0]
    y = y.view(batch_size, 1)
    y_true = y.detach().numpy()
    y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    overall_accuracy += accuracy * batch_size
test_accuracy = overall_accuracy / len(test_helper)
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.5628415300546448


#LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=64, num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, X):
        _, (hn, _) = self.lstm(X)
        hn = hn.permute(1, 0, 2).contiguous()
        return hn.view(X.size(0), 1, self.hidden_dim)

In [ ]:
class NN7(nn.Module):
    def __init__(self, input_dim=64, hidden_dim1=128, hidden_dim2=64, output_dim=1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.LL1 = nn.Linear(self.input_dim, self.hidden_dim1)
        self.LL2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.FL = nn.Linear(self.hidden_dim2, self.output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        batch_size = X.shape[0]
        X = torch.mean(X, 1)
        X = self.LL1(X)
        X = self.sigmoid(X)
        X = self.LL2(X)
        X = self.sigmoid(X)
        X = self.FL(X)
        X = self.sigmoid(X)

        #return X
        return X.view(batch_size, self.output_dim)

In [ ]:
lstm_model = LSTM()
nn_model3 = NN7()

criterion2 = nn.BCELoss(reduction='mean')
optimizer2 = torch.optim.Adam(nn_model3.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    overall_accuracy = 0
    for x, y in train_dloader:
        #print(f'from tloader y: {y.shape}')
        lstm_output = lstm_model(x)
        #print(f'rnn out:  {rnn_output.shape}')
        predicted_y = nn_model3(lstm_output)
        #print(f'pred-y from NN6: {predicted_y.shape}')
        batch_size = x.shape[0]
        #predicted_y = predicted_y.squeeze(1).t()
        #print(f'pred-y from squeeze: {predicted_y.shape}')
        y = y.view(batch_size, 1)
        #print(f'final y from view: {y.shape}')
        loss = criterion2(predicted_y, y)
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()
        y_true = y.detach().numpy()
        y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
        accuracy = accuracy_score(y_true, y_pred)
        overall_accuracy += accuracy * batch_size

    print(f'Epoch: {epoch} --> Accuracy {overall_accuracy/len(train_helper)}')

Epoch: 0 --> Accuracy 0.4931506849315068
Epoch: 1 --> Accuracy 0.4986301369863014
Epoch: 2 --> Accuracy 0.46164383561643835
Epoch: 3 --> Accuracy 0.5287671232876713
Epoch: 4 --> Accuracy 0.4780821917808219
Epoch: 5 --> Accuracy 0.5205479452054794
Epoch: 6 --> Accuracy 0.49726027397260275
Epoch: 7 --> Accuracy 0.5027397260273972
Epoch: 8 --> Accuracy 0.5027397260273972
Epoch: 9 --> Accuracy 0.5123287671232877


In [ ]:
test_accuracy = 0
overall_accuracy = 0
for x, y in test_dloader:
    lstm_output = lstm_model(x)
    predicted_y = nn_model3(lstm_output)
    batch_size = x.shape[0]
    y = y.view(batch_size, 1)
    y_true = y.detach().numpy()
    y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    overall_accuracy += accuracy * batch_size
test_accuracy = overall_accuracy / len(test_helper)
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.5573770491803278


#GRU

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim=300, hidden_dim=64, num_layers=1):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, X):
        _, hn = self.gru(X)
        hn = hn.permute(1, 0, 2).contiguous()
        return hn.view(X.size(0), 1, self.hidden_dim)

In [ ]:
class NN8(nn.Module):
    def __init__(self, input_dim=64, hidden_dim1=128, hidden_dim2=64, output_dim=1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.output_dim = output_dim
        self.LL1 = nn.Linear(self.input_dim, self.hidden_dim1)
        self.LL2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.FL = nn.Linear(self.hidden_dim2, self.output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        batch_size = X.shape[0]
        X = torch.mean(X, 1)
        #X.requires_grad=True
        X = self.LL1(X)
        X = self.sigmoid(X)
        X = self.LL2(X)
        X = self.sigmoid(X)
        X = self.FL(X)
        X = self.sigmoid(X)

        #return X.view(batch_size)
        #return X
        return X.view(batch_size, self.output_dim)

In [ ]:
gru_model = GRU()
nn_model5 = NN8()

criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(nn_model3.parameters(), lr=0.001)

In [ ]:
for epoch in range(10):
    overall_accuracy = 0
    for x, y in train_dloader:
        #print(f'from tloader x: {x.shape}')
        #print(f'from tloader y: {y.shape}')
        gru_output = gru_model(x)
        #print(f'gru out:  {gru_output.shape}')
        predicted_y = nn_model5(gru_output)
        #print(f'pred-y from NN8: {predicted_y.shape}')
        batch_size = x.shape[0]
        #predicted_y = predicted_y.squeeze(1).t()
        #print(f'pred-y from squeeze: {predicted_y.shape}')
        y = y.view(batch_size, 1)
        #print(f'final y from view: {y.shape}')
        loss = criterion(predicted_y, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_true = y.detach().numpy()
        y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
        accuracy = accuracy_score(y_true, y_pred)
        overall_accuracy += accuracy * batch_size

    print(f'Epoch: {epoch} --> Accuracy {overall_accuracy/len(train_helper)}')

Epoch: 0 --> Accuracy 0.5095890410958904
Epoch: 1 --> Accuracy 0.5095890410958904
Epoch: 2 --> Accuracy 0.5095890410958904
Epoch: 3 --> Accuracy 0.5095890410958904
Epoch: 4 --> Accuracy 0.5095890410958904
Epoch: 5 --> Accuracy 0.5095890410958904
Epoch: 6 --> Accuracy 0.5095890410958904
Epoch: 7 --> Accuracy 0.5095890410958904
Epoch: 8 --> Accuracy 0.5095890410958904
Epoch: 9 --> Accuracy 0.5095890410958904


In [ ]:
test_accuracy = 0
overall_accuracy = 0
for x, y in test_dloader:
    gru_output = gru_model(x)
    predicted_y = nn_model5(gru_output)
    batch_size = x.shape[0]
    y = y.view(batch_size, 1)
    y_true = y.detach().numpy()
    y_pred = (predicted_y.detach().numpy() > 0.5).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    overall_accuracy += accuracy * batch_size
test_accuracy = overall_accuracy / len(test_helper)
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.5573770491803278
